- A diagram one of your architectures (probably, your best).

- A description of its architecture, design decisions, hyper-parameters (loss function, batch size, epochs, data size, etc).  You should also include a very brief summary of what you tried as a paragraph about the "big picture."

- Describe two more variants you tried:
  - One architectural variant (i.e., that changes the network graph).
  - Choose a hyperparameter for your "main" variant and pick two other variants of it (e.g. dropout, batch size, etc).  Describe why you picked that variation and what you expected.
  - Plot the losses (whichever loss function you're using) for your "main" architecture, the architectural variant, and the two hyperparameter variants, so four loss graphs.

- Describe in 1-2 paragraphs what you learned.

- To develop your "scientific common sense": Answer the question: why is it bad to validate/test on the training data?

- Include all the four saved model files in the repo in a manner that can be loaded by the testing script.

- If they are too large for github, we will come up with an alternative solution to provide them to us -- contact Vlad if so, before the submission deadline.

## Pre-Report

First we need to check out which of the models performed the best.

# LT2316 H18 Assignment 2

###  Model Diagram

In [1]:
from keras.models import load_model
from keras.utils import plot_model

model = load_model('./models/lstm_simple_embed100.hdf5')
plot_model(model, to_file="./imgs/lstm_simple_embed100.jpg", show_shapes=True)

Using TensorFlow backend.


![Simple](imgs/lstm_simple_embed100.jpg)

### Architecture Overview

After experiementing with many different architectures and LSTM layers, what ended up working the best was a single LSTM layer with 50 nodes. Each word is represented as an integer less than 10000. Some of the other approaches I tried were to have the embedding layer input into two different LSTM layers which in turn seperately predicted either word prediction, or category prediction. This performed worse than other options as well. The other architectures I experimented with are described below, where two LSTM layers are used (passing sequences from first to the second). 

I had originally started with a window size of 5 and then tested a window size of 10. The accuracy was a little better but when using word prediction script the results did not appear as good. The tradeoff of training time was also not worth it as the window size of 10 took almost twice as long to run.

  - **Loss function**: Categorical Cross Entropy for the word prediction since we have one-hot vector encodings and Binary Cross Entropy for the multi-category classification.
  - **Batch Size**: 256
  - **Epochs**: 10 (For final test)
  - **Window size**: 3
  - **Data size**: Using the window size of 10, I created 5572508 training examples from the 591753 captions in the training data. In the test set, 236352 training examples were created from 25014 captions.
  - **Dropout**: dropout was set to 0.1 for the LSTM layer. This performed better than no dropout and 0.5 dropout. It was also not the best test for this as I think with many more epochs it could have performed the same and generalized better.

In [2]:
# Alternative model
model = load_model('./models/lstm_complex_drop1.hdf5')
plot_model(model, to_file="./imgs/lstm_complex_drop1.jpg", show_shapes=True)

### Variant
![Two LSTM Layers](./imgs/lstm_complex_drop1.jpg)

This architectural variant did not seem to perform as well at the simple one layer LSTM model. I experimented with setting the dropout rate which increased the loss. This might not have been fair to do, since I did not run it for many epochs. Increasing the dropout rate should make the model converge slower but overfit less often which I didn't give it the chance to do with such a small number of epochs.

### Hyperparameter Selection  
  
  After experiementing with many different architectures and LSTM layers, what ended up working the best was a single LSTM layer with 50 nodes. I first changed the embedding size to 100 and then for the two other variations a more complex model (seen above) with additional dropout at levels 0.1 and 0.5. I thought that adding another LSTM layer would perform better than the single layer. I am not sure if increasing the epochs or modifying the other hyperparameters would have made a difference.
  
### Model Loss for Different Hyperparameters
![Model Loss](./imgs/model_loss.png)

The final model I went with was the *Simple LSTM 100 Embed*. This was an increase from the base model from 50 node embedding layer to 100. The more complex models (with an additional LSTM layer and dropout) did not perform as well overall.

Code to generate the plot is show below

In [1]:
import matplotlib.pyplot as plt

import pandas as pd

model_simple = pd.read_csv('./results/compare_simple.csv')
model_simple_embed100 = pd.read_csv('./results/compare_embed_100.csv')
model_complex_drop1 = pd.read_csv('./results/compare_complex_embed_100_do1.csv')
model_complex_drop5 = pd.read_csv('./results/compare_complex_embed_100_do5.csv')

# Plot overall loss
plt.plot(model_simple['epoch'], model_simple['loss'], label='Simple LSTM 50 Embed')
plt.plot(model_simple_embed100['epoch'], model_simple_embed100['loss'], label='Simple LSTM 100 Embed')
plt.plot(model_complex_drop1['epoch'], model_complex_drop1['loss'], label='Complex LSTM 100 Embed Dropout 0.1')
plt.plot(model_complex_drop5['epoch'], model_complex_drop5['loss'], label='Complex LSTM 100 Embed Dropout 0.5')

plt.legend(loc='upper right')
plt.title("Overall Cross Entropy Loss")

plt.savefig('./imgs/model_loss.png', bbox_inches='tight')

### Lessons Learned

I learned lots about language models from building this word and category prediction. Also this project was a nice introduction to the Keras functional API. It was interesting to experiment with branching into two seperate LSTM models and combining at the end. The functional API is very simple to use and extremely powerful for branching and rejoining different layers. While Keras makes it easy to train complex models, I would like to dig more deep into some of the theory behind the different layers, and the reasoning behind various hyperparameters. Another thing that I learned was the difference between categorical cross entropy and binary cross entropy. I had originally just used categorical cross entropy for both the word prediction and category prediction.

This project also taught me the importance of starting early on projects. Both the preprocessing of the data and the training time resulted in many hours of debugging and waiting. The process of preparing the text data was challenging but can be easily used for other word based models I use in the future. I would definitely use a smaller window size for prediction and systematically increase it rather than jumping to something high. This would give me the opportunity to experiment more with other hyperparameters because it would reduce the amount of training time. Such as, increasing the dropout rate should make the model converge slower but overfit less often which I didn't give it the chance to do with such a small number of epochs.

### Why is it bad to validate/test on the training data?

It is bad to validate and test on the training data as we will tend to overfit when we eventually was to predict the test set. If we tweak the model too much while validating on the training data we will be not only overfitting in the model, but we will be human overfitting the data by modifying the model. Having a few hypothesis before hand and testing them out would be a better options

In [30]:
m1 = model_simple.iloc[9]
m2 = model_simple_embed100.iloc[9]
m3 = model_complex_drop1.iloc[9]
m4 = model_complex_drop5.iloc[9]
m1.name = "model_simple"
m2.name = "model_simple_embed100"
m3.name = "model_complex_drop1"
m4.name = "model_complex_drop5"

model_df = pd.DataFrame([m1, m2, m3, m4])
model_df[["category_prediction_categorical_accuracy", "category_prediction_acc","word_prediction_categorical_accuracy"]].to_csv("accuracy_res.md", sep="|")